# **Plutchik 8 Emotions**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Libraries

In [2]:
# Installing libraries

!pip install datasets
!pip install transformers
!pip install nltk emoji==0.6.0
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 7.8 MB/s 
     |████████████████████████████████| 101 kB 12.8 MB/s 
     |████████████████████████████████| 115 kB 71.2 MB/s 
     |████████████████████████████████| 212 kB 67.7 MB/s 
     |████████████████████████████████| 141 kB 84.4 MB/s 
     |████████████████████████████████| 596 kB 62.1 MB/s 
     |████████████████████████████████| 127 kB 60.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.2 MB/s 
     |██████████████████████████

In [3]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time

from datasets import load_dataset
from sklearn import metrics
import tensorflow as tf
import tensorflow_addons as tfa

import transformers
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import BertTokenizer, AutoTokenizer
from keras.preprocessing.sequence import pad_sequences

## 2. Dataset

In [4]:
# SemEval dataset

semeval = load_dataset('sem_eval_2018_task_1', 'subtask5.english')
data = semeval.data

label_cols = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

train_set_sem = data["train"].to_pandas()
val_set_sem = data["validation"].to_pandas()
test_set_sem = data["test"].to_pandas()

train_set_sem.drop('ID', inplace=True, axis=1)
val_set_sem.drop('ID', inplace=True, axis=1)
test_set_sem.drop('ID', inplace=True, axis=1)

train_set_sem = train_set_sem.drop(columns=['love', 'optimism', 'pessimism'])
val_set_sem = val_set_sem.drop(columns=['love', 'optimism', 'pessimism'])
test_set_sem = test_set_sem.drop(columns=['love', 'optimism', 'pessimism'])


# Comment/uncomment to train with Sem_Eval train+val together
# frames = [train_set, val_set]
# train_set = pd.concat(frames)
# train_set = train_set.reset_index(drop=True)

print(train_set_sem.shape, val_set_sem.shape, test_set_sem.shape)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/6838 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3259 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/886 [00:00<?, ? examples/s]

Dataset sem_eval_2018_task_1 downloaded and prepared to /root/.cache/huggingface/datasets/sem_eval_2018_task_1/subtask5.english/1.1.0/a7c0de8b805f1988b118882fb289ccfbbeb9085c7820b6f046b5887e234af182. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

(6838, 9) (886, 9) (3259, 9)


In [5]:
# CovidEmo dataset

covid_df = pd.read_csv('/content/drive/MyDrive/CovidEmo/CovidEmo.csv')
covid_df = covid_df.drop(columns=['Unnamed: 0', 'love', 'optimism', 'pessimism'])

covid_df = covid_df.rename(columns={'Tweets': 'Tweet'})

covid_df_train = covid_df.iloc[:860]
covid_df_val = covid_df.iloc[860:1000]
covid_df_test = covid_df.iloc[1000:]

In [6]:
# Combining datasets

train_frames = [train_set_sem, covid_df_train]
val_frames = [val_set_sem, covid_df_val]
test_frames = [test_set_sem, covid_df_test]

train_set = pd.concat(train_frames)
val_set = pd.concat(val_frames)
test_set = pd.concat(test_frames)

train_set = train_set.reset_index(drop=True)
val_set = val_set.reset_index(drop=True)
test_set = test_set.reset_index(drop=True)


# Comment/uncomment to train with train+val sets 

frames = [train_set, val_set]
train_set = pd.concat(frames)
train_set = train_set.reset_index(drop=True)

In [7]:
print(len(train_set))
print(len(val_set))
print(len(test_set))

8724
1026
3695


## 3. Dataset processing

In [8]:
# Tweet pre-process

from emoji import demojize
from nltk.tokenize import TweetTokenizer


tokenizer = TweetTokenizer()


def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token


def normalizeTweet(tweet):
    tokens = tokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = (
        normTweet.replace("cannot ", "can not ")
        .replace("n't ", " n't ")
        .replace("n 't ", " n't ")
        .replace("ca n't", "can't")
        .replace("ai n't", "ain't")
    )
    normTweet = (
        normTweet.replace("'m ", " 'm ")
        .replace("'re ", " 're ")
        .replace("'s ", " 's ")
        .replace("'ll ", " 'll ")
        .replace("'d ", " 'd ")
        .replace("'ve ", " 've ")
    )
    normTweet = (
        normTweet.replace(" p . m .", "  p.m.")
        .replace(" p . m ", " p.m ")
        .replace(" a . m .", " a.m.")
        .replace(" a . m ", " a.m ")
    )

    return " ".join(normTweet.split())

def normalise_dataset(dataset):
  for entry in dataset['Tweet']:
    dataset['Tweet'] = dataset['Tweet'].replace(entry, normalizeTweet(entry))


normalise_dataset(train_set)
normalise_dataset(val_set)
normalise_dataset(test_set)

In [9]:
# Pre-process data

label_cols = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust']

tf.random.set_seed(1234)
#bert_model_name = "bert-base-uncased"
#bert_model_name = "bert-large-uncased"
#bert_model_name = "vinai/bertweet-base"
bert_model_name = "vinai/bertweet-large"

tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
MAX_LEN = 64

def tokenize_sentences(sentences, tokenizer, max_seq_len = 64):
    tokenized_sentences = []

    for sentence in tqdm(sentences):
        tokenized_sentence = tokenizer.encode(sentence, add_special_tokens = True, truncation=True, max_length = max_seq_len)
        
        tokenized_sentences.append(tokenized_sentence)

    return tokenized_sentences

def create_attention_masks(tokenized_and_padded_sentences):
    attention_masks = []

    for sentence in tokenized_and_padded_sentences:
        att_mask = [int(token_id > 0) for token_id in sentence]
        attention_masks.append(att_mask)

    return np.asarray(attention_masks)

train_input_ids = tokenize_sentences(train_set['Tweet'], tokenizer, MAX_LEN)
print(train_input_ids)
train_inputs = pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
train_masks = create_attention_masks(train_inputs)

val_input_ids = tokenize_sentences(val_set['Tweet'], tokenizer, MAX_LEN)
val_inputs = pad_sequences(val_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
val_masks = create_attention_masks(val_inputs)

test_input_ids = tokenize_sentences(test_set['Tweet'], tokenizer, MAX_LEN)
test_inputs = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
test_masks = create_attention_masks(test_inputs)

train_labels =  train_set[label_cols].values
val_labels = val_set[label_cols]
test_labels = test_set[label_cols].values

  0%|          | 0/8724 [00:00<?, ?it/s]

[[0, 17, 48, 305, 17649, 16, 10, 159, 3207, 15, 10, 936, 47, 189, 393, 33, 128, 479, 12181, 11392, 479, 849, 25331, 38591, 849, 23240, 4128, 849, 605, 17649, 2], [0, 25216, 47, 2845, 7, 109, 146, 686, 24, 817, 47, 849, 27333, 479, 2], [0, 1039, 47955, 24, 67, 2607, 14, 5, 1647, 9, 1485, 4880, 16, 35335, 479, 993, 9, 1585, 384, 108, 9814, 128, 29, 310, 1765, 21, 26388, 2156, 27785, 849, 43070, 26063, 2], [0, 47127, 5, 2019, 98, 14, 47, 64, 5909, 190, 619, 5, 32749, 1258, 9, 1124, 479, 128, 111, 111, 1655, 208, 479, 24040, 4832, 16319, 1215, 9021, 35, 2], [0, 2387, 25537, 4832, 24, 128, 29, 8578, 14, 52, 64, 75, 8921, 142, 52, 33, 7241, 1975, 42892, 479, 849, 1334, 24786, 849, 9502, 8331, 4892, 4311, 2], [0, 3084, 53, 14, 128, 29, 98, 11962, 479, 83, 1872, 257, 21, 1153, 9152, 59, 2356, 137, 53, 20075, 1147, 69, 66, 1717, 13104, 2], [0, 8275, 47, 206, 5868, 33, 5, 1472, 13, 16257, 18049, 27833, 17487, 2], [0, 27110, 1264, 2459, 23523, 7587, 12746, 868, 16, 295, 75, 37, 17487, 30857, 2352

  0%|          | 0/1026 [00:00<?, ?it/s]

  0%|          | 0/3695 [00:00<?, ?it/s]

In [10]:
print(test_input_ids[3500])
print(tokenizer.decode(test_input_ids[3500]))

[0, 713, 16, 10, 8018, 235, 27785, 252, 236, 201, 7, 912, 37980, 15647, 142, 44, 48, 24, 16, 295, 75, 2105, 1666, 44, 46, 27785, 178, 187, 77, 222, 5, 13387, 555, 776, 18727, 17487, 44454, 42703, 2]
<s>This is a joke right! They want us to stop vaccinating because “ it isn't fair... ”! And since when did the WHO become economic advisors? HTTPURL</s>


In [ ]:
# Create datasets

BATCH_SIZE = 16
NR_EPOCHS = 3

def create_dataset(data_tuple, epochs=1, batch_size=32, buffer_size=10000, train=True):
    dataset = tf.data.Dataset.from_tensor_slices(data_tuple)
    if train:
        dataset = dataset.shuffle(buffer_size=buffer_size)
    dataset = dataset.repeat(epochs)
    dataset = dataset.batch(batch_size)
    if train:
        dataset = dataset.prefetch(1)
    return dataset

train_dataset = create_dataset((train_inputs, train_masks, train_labels), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)
validation_dataset = create_dataset((val_inputs, val_masks, val_labels), epochs=NR_EPOCHS, batch_size=BATCH_SIZE)
test_dataset = create_dataset((test_inputs, test_masks), batch_size=BATCH_SIZE, train=False, epochs=1)

## 4. Plutchik model

In [12]:
from transformers import TFBertModel, TFAutoModel, AutoTokenizer, TFRobertaModel


class AutoClassifier(tf.keras.Model):
      def __init__(self, bert: TFRobertaModel, num_classes: int):
        super().__init__()
        self.bert = bert
        self.classifier = tf.keras.layers.Dense(num_classes, activation='sigmoid')
        self.pre_classifier = tf.keras.layers.Dense(2048, activation='relu')

      @tf.function
      def call(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
        outputs = self.bert(input_ids,
                               attention_mask=attention_mask,
                               token_type_ids=token_type_ids,
                               position_ids=position_ids,
                               head_mask=head_mask)

        cls_output = outputs[1]
        cls_output = self.pre_classifier(cls_output)
        cls_output = self.classifier(cls_output)
                
        return cls_output


class BertClassifier(tf.keras.Model):    
    def __init__(self, bert: TFBertModel, num_classes: int):
        super().__init__()

        self.bert = bert
        self.classifier = tf.keras.layers.Dense(num_classes, activation='sigmoid')
        
    @tf.function
    def call(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
        outputs = self.bert(input_ids,
                               attention_mask=attention_mask,
                               token_type_ids=token_type_ids,
                               position_ids=position_ids,
                               head_mask=head_mask)
        cls_output = outputs[1]
        cls_output = self.classifier(cls_output)
                
        return cls_output

print(len(label_cols))

model = AutoClassifier(TFAutoModel.from_pretrained(bert_model_name), len(label_cols)) # Use with BERTweet
#model = BertClassifier(TFBertModel.from_pretrained(bert_model_name), len(label_cols)) # Use with BERT

8


Some layers from the model checkpoint at vinai/bertweet-large were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFRobertaModel were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['roberta/pooler/dense/bias:0', 'roberta/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
print(train_inputs[:1])

model(train_inputs[:1],train_masks[:1]).numpy()
model.summary()

[[    0    17    48   305 17649    16    10   159  3207    15    10   936
     47   189   393    33   128   479 12181 11392   479   849 25331 38591
    849 23240  4128   849   605 17649     2     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]]
Model: "auto_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tf_roberta_model (TFRoberta  multiple                 355359744 
 Model)                                                          
                                                                 
 dense (Dense)               multiple                  16392     
                                                                 
 dense_1 (Dense)             multiple                  2099200   
                                                 

## 5. Training

In [14]:
INIT_LR = 2e-7
MAX_LR = 2e-5

# Loss Function
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)
train_loss = tf.keras.metrics.Mean(name='train_loss')
validation_loss = tf.keras.metrics.Mean(name='test_loss')

# Cyclical Learning Rate Optimizer
clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=INIT_LR,
    maximal_learning_rate=MAX_LR,
    scale_fn=lambda x: 1/(2.**(x-1)), 
    step_size=4 * (len(train_dataset) // BATCH_SIZE)
)

optimizer = tf.keras.optimizers.Adam(clr)
#optimizer = tf.keras.optimizers.Adam(learning_rate=2e-6)

# Metrics
train_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]
validation_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]
train_loss_results = []

# Training step
@tf.function
def train_step(model, token_ids, masks, labels):
    with tf.GradientTape() as tape:
        predictions = model(token_ids, attention_mask = masks, training=True)
        loss_value = loss_fn(labels, predictions)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_loss(loss_value)

    for i, auc in enumerate(train_auc_metrics):
        auc.update_state(labels[:,i], predictions[:,i])


# Validation step
@tf.function
def validation_step(model, token_ids, masks, labels):
    predictions = model(token_ids, attention_mask=masks, training=False)
    val_loss = loss_fn(labels, predictions)
    validation_loss(val_loss)

    for i, auc in enumerate(validation_auc_metrics):
        auc.update_state(labels[:,i], predictions[:,i])

# Training loop
def train(model, train_dataset, val_dataset, epochs):
  for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    for step, (token_ids, masks, labels) in enumerate(tqdm(train_dataset)):

      train_step(model, token_ids, masks, labels)
      if step % 100 == 0:
        train_loss_results.append(train_loss.result())
        print(f'\nTrain Step: {step}, Loss: {train_loss.result()}')
        for i, label_name in enumerate(label_cols):
          print(f"{label_name} roc_auc {train_auc_metrics[i].result()}")
          train_auc_metrics[i].reset_states()

    for i, (token_ids, masks, labels) in enumerate(tqdm(val_dataset)):
            validation_step(model, token_ids, masks, labels)

    print(f'\nEpoch {epoch+1}, Validation Loss: {validation_loss.result()}, Time: {time.time()-start_time}\n')

    for i, label_name in enumerate(label_cols):
      print(f"{label_name} roc_auc {validation_auc_metrics[i].result()}")
      validation_auc_metrics[i].reset_states()

    print('\n')
    model.save_weights("/content/drive/MyDrive/Models/covid{}.h5".format(epoch+1))

train(model, train_dataset, validation_dataset, epochs=3)


Start of epoch 0


  0%|          | 0/1636 [00:00<?, ?it/s]


Train Step: 0, Loss: 0.6865603923797607
anger roc_auc 0.2890625
anticipation roc_auc 0.0
disgust roc_auc 0.3174603283405304
fear roc_auc 0.699999988079071
joy roc_auc 0.59375
sadness roc_auc 0.6545454263687134
surprise roc_auc 0.0
trust roc_auc 0.0

Train Step: 100, Loss: 0.5884073376655579
anger roc_auc 0.49270251393318176
anticipation roc_auc 0.5021713376045227
disgust roc_auc 0.5062223672866821
fear roc_auc 0.5411263108253479
joy roc_auc 0.49857771396636963
sadness roc_auc 0.5015260577201843
surprise roc_auc 0.5186856389045715
trust roc_auc 0.43029624223709106

Train Step: 200, Loss: 0.5425015091896057
anger roc_auc 0.5246332287788391
anticipation roc_auc 0.5323246717453003
disgust roc_auc 0.5414924621582031
fear roc_auc 0.5212670564651489
joy roc_auc 0.5576802492141724
sadness roc_auc 0.49223893880844116
surprise roc_auc 0.499668687582016
trust roc_auc 0.5211048126220703

Train Step: 300, Loss: 0.5164440274238586
anger roc_auc 0.6941335201263428
anticipation roc_auc 0.587428748607

  0%|          | 0/193 [00:00<?, ?it/s]


Epoch 1, Validation Loss: 0.2321508675813675, Time: 760.6603648662567

anger roc_auc 0.961212158203125
anticipation roc_auc 0.8603879809379578
disgust roc_auc 0.9459149837493896
fear roc_auc 0.9564316868782043
joy roc_auc 0.9719103574752808
sadness roc_auc 0.9270685911178589
surprise roc_auc 0.9112189412117004
trust roc_auc 0.8635075688362122



Start of epoch 1


...


Epoch 2, Validation Loss: 0.21313460171222687, Time: 769.9206779003143

anger roc_auc 0.9752690196037292
anticipation roc_auc 0.886517345905304
disgust roc_auc 0.9611740112304688
fear roc_auc 0.971289336681366
joy roc_auc 0.9848384857177734
sadness roc_auc 0.9488596320152283
surprise roc_auc 0.9457114338874817
trust roc_auc 0.897732675075531



Start of epoch 2


  0%|          | 0/1636 [00:00<?, ?it/s]


Train Step: 0, Loss: 0.2859123647212982
anger roc_auc 0.9646149277687073
anticipation roc_auc 0.8773465752601624
disgust roc_auc 0.9353061318397522
fear roc_auc 0.9573061466217041
joy roc_auc 0.9830456972122192
sadness roc_auc 0.9248369336128235
surprise roc_auc 0.8443799018859863
trust roc_auc 0.8876010775566101

Train Step: 100, Loss: 0.28375014662742615
anger roc_auc 0.9724132418632507
anticipation roc_auc 0.8662773370742798
disgust roc_auc 0.9491526484489441
fear roc_auc 0.970102071762085
joy roc_auc 0.9810178279876709
sadness roc_auc 0.9281099438667297
surprise roc_auc 0.9037697315216064
trust roc_auc 0.9092609286308289

Train Step: 200, Loss: 0.2816375494003296
anger roc_auc 0.9760164618492126
anticipation roc_auc 0.8762781023979187
disgust roc_auc 0.9479577541351318
fear roc_auc 0.9597809314727783
joy roc_auc 0.9843953847885132
sadness roc_auc 0.9330616593360901
surprise roc_auc 0.9177775979042053
trust roc_auc 0.9235395789146423

Train Step: 300, Loss: 0.2797563970088959
anger

  0%|          | 0/193 [00:00<?, ?it/s]


Epoch 3, Validation Loss: 0.20309770107269287, Time: 770.0567834377289

anger roc_auc 0.9783721566200256
anticipation roc_auc 0.9003963470458984
disgust roc_auc 0.9674682021141052
fear roc_auc 0.9763153195381165
joy roc_auc 0.9864920973777771
sadness roc_auc 0.9541517496109009
surprise roc_auc 0.9501896500587463
trust roc_auc 0.9049577116966248




In [24]:
model.save_weights("/content/drive/MyDrive/Models/train_test_model.h5")

## 6. Evaluate

In [16]:
df_result = test_set.copy(deep=True)
df_result = df_result.drop_duplicates('Tweet', keep='first')

df_result[label_cols] = 0.5
df_result.set_index('Tweet', inplace=True)

df_result.head()
test_set.head()

,Tweet,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,@USER @USER Dont worry Indian army is on its w...,True,True,False,False,False,False,False,True
1,"Academy of Sciences , eschews the normally sob...",False,False,True,False,False,False,False,False
2,I blew that opportunity - __ - #mad,True,False,True,False,False,True,False,False
3,This time in 2 weeks I will be 30 ... :sad_but...,False,False,False,False,True,True,False,False
4,#Deppression is real . Partners w / #depressed...,False,False,False,True,False,True,False,False


In [17]:
from sklearn.metrics import multilabel_confusion_matrix, classification_report, hamming_loss, f1_score, accuracy_score, jaccard_score

#test_steps = len(test_set) // BATCH_SIZE

test_auc_metrics = [tf.keras.metrics.AUC() for i in range(len(label_cols))]

for i, (token_ids, masks) in enumerate(tqdm(test_dataset)):
    labels = test_labels[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
    labels = tf.dtypes.cast(labels, tf.float32)
    
    sample_ids = test_set.iloc[i*BATCH_SIZE:(i+1)*BATCH_SIZE]['Tweet']
    predictions = model(token_ids, attention_mask=masks).numpy()

    df_result.loc[sample_ids, label_cols] = predictions

    for i, auc in enumerate(test_auc_metrics):
      auc.update_state(labels[:,i], predictions[:,i])

  0%|          | 0/231 [00:00<?, ?it/s]

In [18]:
for i, label_name in enumerate(label_cols):
    print(f"{label_name} roc_auc {test_auc_metrics[i].result()}")
    test_auc_metrics[i].reset_states()

anger roc_auc 0.9351423382759094
anticipation roc_auc 0.7940816879272461
disgust roc_auc 0.9109113812446594
fear roc_auc 0.9281235933303833
joy roc_auc 0.9526517987251282
sadness roc_auc 0.9031708240509033
surprise roc_auc 0.790091872215271
trust roc_auc 0.8073334693908691


In [ ]:
y_true = test_labels

best_thresh = 0
best_acc = 0
for thresh in np.arange(0.1, 1, 0.01):
  y_pred = df_result[label_cols].values
  y_pred = np.array([[1 if i > thresh else 0 for i in j] for j in y_pred])
  acc = jaccard_score(y_true, y_pred, average='samples')
  
  if acc > best_acc:
    best_thresh = thresh
    best_acc = acc
    
print(best_thresh)

In [20]:
from sklearn.metrics import accuracy_score, jaccard_score

y_true = test_labels
y_pred = df_result[label_cols].values

thresh = best_thresh
y_pred = np.array([[1 if i > thresh else 0 for i in j] for j in y_pred])

#print(f1_score(y_true, y_pred, average=None))

print('Micro F1',f1_score(y_true, y_pred,average='micro'))
print('Macro F1',f1_score(y_true, y_pred,average='macro'))
print('Accuracy', jaccard_score(y_true, y_pred, average='samples'))

Micro F1 0.7320867614061332
Macro F1 0.5943764094013879
Accuracy 0.6519079837618403


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
